<a href="https://colab.research.google.com/github/h4ck4l1/datasets/blob/main/NLP_with_RNN_and_Attention/Bahdanau_Attention_with_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()
import os,warnings
os.environ["TF_MIN_LOG_LEVEL"] = "3"
warnings.filterwarnings("ignore")
import tensorflow as tf
from tensorflow import keras
import plotly.graph_objects as go
import plotly.io as pio
from zipfile import ZipFile
pio.templates.default = "plotly_dark"
tf.get_logger().setLevel("ERROR")

In [2]:
url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"

In [3]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

In [4]:
with tf.device("/job:localhost"):
    file_path = keras.utils.get_file(fname="spa-eng.zip",origin=url,extract=True)
    with ZipFile(file_path,"r") as f:
        f.extractall("spa-eng")
    with open("spa-eng/spa-eng/spa.txt","r") as f:
        text = f.read()
    text = text.replace("¡","").replace("¿","")
    text = [line.split("\t") for line in text.splitlines()]
    en_text,es_text = zip(*text)
    total_size = len(en_text)

2638744/2638744 [==============================] - 0s 0us/step


In [5]:
def get_layers(vocab_size=1000,seq_length=50):
    en_vec_layer = keras.layers.TextVectorization(vocab_size,output_sequence_length=50)
    es_vec_layer = keras.layers.TextVectorization(vocab_size,output_sequence_length=50)
    en_vec_layer.adapt(en_text)
    es_vec_layer.adapt([f"soseq {s} eoseq" for s in es_text])
    return en_vec_layer,es_vec_layer

In [6]:
def get_data(es_vec_layer,train_size=100_000):

    # if full:
    train_size = slice(None,100_000)
    valid_size = slice(100_000,None)
    # else:
    #     train_size = slice(None,1000)
    #     valid_size = slice(1000,1500)

    tsize = train_size.stop
    vsize = (valid_size.stop - valid_size.start) if valid_size.stop else (len(en_text) - tsize)
    X_train = en_vec_layer(tf.constant(en_text[train_size]))
    X_valid = en_vec_layer(tf.constant(en_text[valid_size]))
    X_dec_train = es_vec_layer(tf.constant([f"soseq {s}" for s in es_text[train_size]]))
    X_dec_valid = es_vec_layer(tf.constant([f"soseq {s}" for s in es_text[valid_size]]))
    y_train = es_vec_layer(tf.constant([f"{s} eoseq" for s in es_text[train_size]]))
    y_valid = es_vec_layer(tf.constant([f"{s} eoseq" for s in es_text[valid_size]]))

    return (X_train,X_dec_train),y_train,(X_valid,X_dec_valid),y_valid,tsize,vsize

In [7]:
# @title
# def get_model(vocab_size=1000,embed_size=128):

#     encoder_inputs = keras.layers.Input(shape=(),dtype=tf.string)
#     decoder_inputs = keras.layers.Input(shape=(),dtype=tf.string)
#     en_vec_out = en_vec_layer(encoder_inputs)
#     es_vec_out = es_vec_layer(decoder_inputs)
#     en_embed = keras.layers.Embedding(vocab_size,embed_size,mask_zero=True)
#     es_embed = keras.layers.Embedding(vocab_size,embed_size,mask_zero=True)
#     en_embed_out = en_embed(en_vec_out)
#     es_embed_out = es_embed(es_vec_out)
#     encoder = keras.layers.Bidirectional(keras.layers.LSTM(256,return_state=True,return_sequences=True))
#     decoder = keras.layers.LSTM(512,return_sequences=True)
#     encoder_out,*encoder_state = encoder(en_embed_out)
#     initial_state = [tf.concat(encoder_state[::2],axis=-1),tf.concat(encoder_state[1::2],axis=-1)]
#     decoder_out = decoder(es_embed_out,initial_state=initial_state)
#     attention = keras.layers.Attention()
#     attention_out = attention([decoder_out,encoder_out])
#     out_layer = keras.layers.Dense(vocab_size,"softmax")
#     out = out_layer(attention_out)
#     return keras.Model(inputs=[encoder_inputs,decoder_inputs],outputs=[out])


In [8]:
class AttentionModel(keras.Model):

    def __init__(self,vocab_size=1000,embed_size=128,**kwargs):

        super(AttentionModel,self).__init__(**kwargs)
        self.en_embed = keras.layers.Embedding(vocab_size,embed_size,mask_zero=True)
        self.es_embed = keras.layers.Embedding(vocab_size,embed_size,mask_zero=True)
        self.encoder = keras.layers.Bidirectional(keras.layers.LSTM(256,return_state=True,return_sequences=True))
        self.decoder = keras.layers.LSTM(512,return_sequences=True)
        self.attention = keras.layers.Attention()
        self.out = keras.layers.Dense(vocab_size,"softmax")

    def call(self,inputs):

        encoder_inputs = inputs[0]
        decoder_inputs = inputs[1]
        en_embed_out = self.en_embed(encoder_inputs)
        es_embed_out = self.es_embed(decoder_inputs)
        encoder_out,*encoder_state = self.encoder(en_embed_out)
        encoder_state = [tf.concat(encoder_state[::2],axis=-1),tf.concat(encoder_state[1::2],axis=-1)]
        decoder_out = self.decoder(es_embed_out,initial_state=encoder_state)
        attention_out = self.attention([decoder_out,encoder_out])
        return self.out(attention_out)

In [9]:
with strategy.scope():

    en_vec_layer,es_vec_layer = get_layers()
    epochs = 20
    X_train,y_train,X_valid,y_valid,train_size,valid_size = get_data(es_vec_layer)
    model = AttentionModel()
    BATCH_SIZE = 50*8
    steps_per_epoch = train_size//BATCH_SIZE
    validation_steps = valid_size//BATCH_SIZE
    num_train_steps = steps_per_epoch * epochs
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer="nadam",
        metrics=["accuracy"],
        steps_per_execution=25
    )

In [10]:
model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=BATCH_SIZE,
    validation_data=(X_valid,y_valid),
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

Epoch 1/20
250/250 [==============================] - 41s 163ms/step - loss: 4.4972 - accuracy: 0.2249 - val_loss: 4.2670 - val_accuracy: 0.2757
Epoch 2/20
250/250 [==============================] - 5s 20ms/step - loss: 3.3085 - accuracy: 0.3959 - val_loss: 3.3398 - val_accuracy: 0.3772
Epoch 3/20
250/250 [==============================] - 5s 20ms/step - loss: 2.4976 - accuracy: 0.4986 - val_loss: 2.7221 - val_accuracy: 0.4492
Epoch 4/20
250/250 [==============================] - 5s 19ms/step - loss: 1.9880 - accuracy: 0.5647 - val_loss: 2.3995 - val_accuracy: 0.4907
Epoch 5/20
250/250 [==============================] - 5s 20ms/step - loss: 1.7108 - accuracy: 0.6090 - val_loss: 2.2318 - val_accuracy: 0.5134
Epoch 6/20
250/250 [==============================] - 5s 19ms/step - loss: 1.5416 - accuracy: 0.6427 - val_loss: 2.1148 - val_accuracy: 0.5334
Epoch 7/20
250/250 [==============================] - 5s 20ms/step - loss: 1.4262 - accuracy: 0.6663 - val_loss: 2.0285 - val_accuracy: 0.54

In [11]:
def translate(sentence:str,beam_width=3):

    translation_list = []
    X_inp = en_vec_layer(tf.constant([sentence]))
    X_dec_inp = es_vec_layer(tf.constant(["soseq"]))
    outputs = model.predict((X_inp,X_dec_inp),verbose=0)[0,0]
    top_k_values,top_k_indices = tf.math.top_k(outputs,beam_width)
    for i in range(beam_width):
        translation = es_vec_layer.get_vocabulary()[top_k_indices[i]]
        pred_proba = tf.math.log(top_k_values[i])
        for word_id in range(1,50):
            X_dec_inp = es_vec_layer(tf.constant(["soseq "+translation]))
            outputs = model.predict((X_inp,X_dec_inp),verbose=0)[0,word_id]
            top_1_val,top_1_ind = tf.math.top_k(outputs,1)
            pred_word = es_vec_layer.get_vocabulary()[top_1_ind[0]]
            if pred_word == "eoseq":
                break
            pred_proba += tf.math.log(top_1_val[0])
            translation += " " + pred_word
        translation_list.append((pred_proba,translation))

    return translation_list

In [12]:
translate("I love Soccer")

[(<tf.Tensor: shape=(), dtype=float32, numpy=-1.926172>,
  'me encanta el fútbol'),
 (<tf.Tensor: shape=(), dtype=float32, numpy=-3.100997>, 'yo amo el fútbol'),
 (<tf.Tensor: shape=(), dtype=float32, numpy=-7.6268578>,
  'le encanta me el fútbol')]